# **模型评估**

在本 Notebook 中，您将学习：

- 什么是模型评估？
- 如何在 ML.NET 中评估模型
- 使用交叉验证训练和评估模型
- 模型可解释性
- 如何改进你的模型

## **什么是模型评估？**

训练是将算法应用于历史数据以创建准确表示该数据的模型的过程。 然后将该模型用于新数据以进行预测。

模型评估是使用指标来量化模型在数据中学习模式的效率并将这些学习应用于新的和看不见的数据的过程。

## **如何在 ML.NET 中评估模型**

### **引用 ML.NET Daily Nuget**

In [1]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json

### **安装相关依赖**

In [1]:
#r "nuget: Microsoft.ML.AutoML, 0.20.0-preview.22356.1"
#r "nuget: Microsoft.Data.Analysis, 0.20.0-preview.22356.1"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22356.1 Microsoft.ML.AutoML, 0.20.0-preview.22356.1

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `Microsoft.ML.AutoML.Interactive.dll`

Reference packages with `using` statements

In [1]:
using System.Text.Json;
using Microsoft.Data.Analysis;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers.FastTree;
using static Microsoft.ML.Transforms.OneHotEncodingEstimator;

### **下载或查找数据**
以下代码尝试在几个已知位置找到数据文件，或者它将从已知的 GitHub 位置下载它。

In [1]:
using System;
using System.IO;
using System.Net;

string EnsureDataSetDownloaded(string fileName)
{

	// This is the path if the repo has been checked out.
	var filePath = Path.Combine(Directory.GetCurrentDirectory(),"data", fileName);

	if (!File.Exists(filePath))
	{
		// This is the path if the file has already been downloaded.
		filePath = Path.Combine(Directory.GetCurrentDirectory(), fileName);
	}

	if (!File.Exists(filePath))
	{
		using (var client = new WebClient())
		{
			client.DownloadFile($"https://raw.githubusercontent.com/dotnet/csharp-notebooks/main/machine-learning/data/{fileName}", filePath);
		}
		Console.WriteLine($"Downloaded {fileName}  to : {filePath}");
	}
	else
	{
		Console.WriteLine($"{fileName} found here: {filePath}");
	}

	return filePath;
}

Train Data Path: C:\dev\csharp-notebooks\machine-learning\data\taxi-fare.csv


In [1]:
var trainDataPath = EnsureDataSetDownloaded("taxi-fare.csv");
var df = DataFrame.LoadCsv(trainDataPath);

加载数据后，使用 `Head` 方法预览前五行。

In [1]:
df.Head(5)

index,vendor_id,rate_code,passenger_count,trip_time_in_secs,trip_distance,payment_type,fare_amount
0,CMT,1,1,1271,3.8,CRD,17.5
1,CMT,1,1,474,1.5,CRD,8
2,CMT,1,1,637,1.4,CRD,8.5
3,CMT,1,1,181,0.6,CSH,4.5
4,CMT,1,1,661,1.1,CRD,8.5


### **初始化 MLContext**

所有 ML.NET 操作都从 [MLContext](https://docs.microsoft.com/dotnet/api/microsoft.ml.mlcontext) 类开始。 初始化 mlContext 会创建一个新的 ML.NET 环境，该环境可以在模型创建工作流对象之间共享。 从概念上讲，它类似于 Entity Framework 中的 DBContext。

In [1]:
var mlContext = new MLContext();

### 将数据拆分为训练集、验证集和测试集

原始数据集分为三个子集：训练、验证和测试。 **train** 集用于学习数据模式。 **validation** 集用于优化模型超参数。 **test** 集用于使用回归任务的评估指标来评估模型的性能。

在这种情况下，80% 的数据用于由“testFraction”参数定义的训练。剩下的 20% 被分成两半，用作验证和测试集。

#### 为什么不使用整个数据集？

尽管通常建议为您的培训师提供更多可以从中学习的示例，但您不希望模型仅在历史数据上表现良好。相反，您正在寻找一个模型，该模型可以从历史数据中学习，并对新的和未见的数据进行概括或做出准确的预测。

您在训练过程中遇到的一些常见问题是过拟合和欠拟合。欠拟合意味着选择的训练器不足以拟合训练数据集，通常会导致训练过程中的高损失和测试数据集的低分数/指标。要解决此问题，您需要选择更强大的模型或执行更多特征工程。过度拟合则相反，当模型对训练数据学习得很好时就会发生这种情况。这通常会导致训练期间的低损失度量，但测试数据集的高损失。

这些概念的一个很好的类比是为考试而学习。假设您提前知道问题和答案。学习后，您参加考试并获得满分。好消息！但是，当您再次参加考试时，重新安排了问题并且措辞略有不同，您会得到较低的分数。这表明您记住了答案，实际上并没有学习您正在测试的概念。这是过拟合的一个例子。拟合不足是相反的情况，即您获得的学习材料不能准确地代表您在考试中所评估的内容。结果，您求助于猜测答案，因为您没有足够的知识来正确回答。

In [1]:
var trainTestData = mlContext.Data.TrainTestSplit(df,testFraction:0.2);
var validationTestData = mlContext.Data.TrainTestSplit(trainTestData.TestSet,testFraction:0.5);

In [1]:
var trainSet = trainTestData.TrainSet;
var validationSet = validationTestData.TrainSet;
var testSet = validationTestData.TestSet;

### 创建训练管道

对于此数据集，应用了以下转换：

- `OneHotEncoding` 将分类值转换为数值
- `ReplaceMissingValues` 顾名思义就是替换任何缺失值。
- `Concatenate` 获取所有特征并创建一个特征向量

AutoML 用于定义回归实验，使用“fare_amount”列作为预测列或标签列。

In [1]:
var pipeline = 
	mlContext.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair(@"vendor_id", @"vendor_id"), new InputOutputColumnPair(@"payment_type", @"payment_type")},outputKind: OutputKind.Binary)
		.Append(mlContext.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair(@"rate_code", @"rate_code"), new InputOutputColumnPair(@"passenger_count", @"passenger_count"), new InputOutputColumnPair(@"trip_time_in_secs", @"trip_time_in_secs"), new InputOutputColumnPair(@"trip_distance", @"trip_distance") }))
        .Append(mlContext.Transforms.Concatenate(@"Features", new[] { @"vendor_id", @"payment_type", @"rate_code", @"passenger_count", @"trip_time_in_secs", @"trip_distance" }))
        .Append(mlContext.Auto().Regression(labelColumnName: "fare_amount"));

### **配置实验**

使用 AutoML 将我们的实验配置为使用您刚刚定义的管道进行 60 秒的训练。

默认情况下，AutoML 使用您要优化的评估指标评估它训练的模型。在这种情况下，它是 R-Squared，它是通过将实际值 `fare_amount` 与预测值 `Score` 进行比较来计算的。

评估指标高度依赖于任务。对于回归，一些常见的指标包括：

- 平均绝对误差 (MAE)
- 均方误差 (MSE)
- 均方根误差 (RMSE)
- R平方

您的数据集和您尝试实现的目标对您的指标选择有很大影响。如果您的数据集中有异常值，它可能会扭曲您的预测。 MAE、MSE 和 RMSE 计算预测数据点和实际数据点之间的距离。所有这些度量对异常值都很敏感，因此如果您的数据集中有异常值，它们将显示在您的指标中。 R-Squared 计算实际值和预测值之间的相关性。但是，随着您添加更多数据点，您的 R-Squared 可能会继续增加，给人一种错误印象，即具有高 R-Squared 值的模型具有良好的预测能力。高 R-Squared 值的结果有时可能表明过度拟合。

In [1]:
var experiment = 
	mlContext.Auto().CreateExperiment()
		.SetPipeline(pipeline)
        .SetTrainingTimeInSeconds(60)
        .SetDataset(trainSet, validationSet)
        .SetEvaluateMetric(RegressionMetric.RSquared, "fare_amount", "Score");

### **运行实验**

In [1]:
var result = await experiment.RunAsync();

### **查看最佳模型的评估指标**

In [1]:
$"R-Squared: {result.Metric}"

R-Squared: 0.9329530384286037

请注意，在训练期间，评估指标是使用验证集计算的。 要查看您的模型在新数据上的表现如何，请根据测试集评估其性能。

首先使用训练结果中的“模型”属性获取最佳模型。 然后，使用 `Transform` 方法使用模型对测试数据集进行预测。

In [1]:
ITransformer bestModel = result.Model;
var predictions = bestModel.Transform(testSet);

检查前几个预测（`Score` 列）并将它们与实际值（`fare_amount` 列）进行比较。 然后，计算它们之间的差异。

In [1]:
var actual = predictions.GetColumn<float>("fare_amount");
var predicted = predictions.GetColumn<float>("Score");

var compare = 
	actual
		.Zip(predicted,(actual,pred) => new {Actual=actual, Predicted=pred, Difference=actual-pred})
		.Take(5);

compare

index,Actual,Predicted,Difference
0,24.5,23.087162,1.412838
1,9.5,8.993666,0.5063343
2,4.5,4.808011,-0.30801105
3,8,7.994398,0.005601883
4,52,52.039684,-0.039684296


只需快速比较前几个值，您就可以看到预测通常与实际金额相差几美分。

使用 ML.NET，您不必手动计算模型的评估指标。 ML.NET 为其支持的每个机器学习任务提供了一个内置的“评估”方法。 使用回归任务的 `Evaluate` 方法计算测试集的评估指标，其中 `fare_amount` 列是实际值， `Score` 列是预测值。

In [1]:
var evaluationMetrics = mlContext.Regression.Evaluate(predictions,"fare_amount", "Score");

Using the `Evaluate` method not only calculates the metric you optimized for during training (R-Squared), but also all the metrics for the regression task. 

In [1]:
evaluationMetrics

MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
0.6107270253580241,6.673910566709432,2.58339129183123,6.673910534194763,0.9277130209892651


### 使用 R-Squared 评估模型

尽管您有多个指标可供选择，但在训练模型时，您已针对 R-Squared 进行了优化。 R 平方 (R2) 或确定系数将模型的预测能力表示为介于 -inf 和 1.00 之间的值。 1.00 表示完美契合，契合度可以任意差，因此得分可能为负数。 0.00 分表示模型正在猜测标签的预期值。 负 R2 值表示拟合不遵循数据的趋势，并且模型的性能比随机猜测差。 这仅适用于非线性回归模型或约束线性回归。 R2 衡量实际测试数据值与预测值的接近程度。


有关其他评估指标的详细信息，请参阅有关如何[使用指标评估您的 ML.NET 模型](https://docs.microsoft.com/dotnet/machine-learning/resources/metrics) 的指南。

## 使用交叉验证训练和评估模型

### 什么是交叉验证？

交叉验证是一种训练和模型评估技术，它将数据分成几个分区并在这些分区上训练多个模型。 这种技术通过保留训练过程中的数据来提高模型的鲁棒性。 除了提高未见观察的性能外，在数据受限的环境中，它还可以成为使用较小数据集训练模型的有效工具。

### 使用交叉验证训练模型

首先初始化“MLContext”。

In [1]:
var cvMLContext = new MLContext();

然后，定义您的管道。 在这种情况下，使用实际的训练器而不是 AutoML 中的“SweepableEstimator”。

In [1]:
var cvMLPipeline = 
	cvMLContext.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair(@"vendor_id", @"vendor_id"), new InputOutputColumnPair(@"payment_type", @"payment_type")},outputKind: OutputKind.Binary)
		.Append(cvMLContext.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair(@"rate_code", @"rate_code"), new InputOutputColumnPair(@"passenger_count", @"passenger_count"), new InputOutputColumnPair(@"trip_time_in_secs", @"trip_time_in_secs"), new InputOutputColumnPair(@"trip_distance", @"trip_distance") }))
        .Append(cvMLContext.Transforms.Concatenate(@"Features", new[] { @"vendor_id", @"payment_type", @"rate_code", @"passenger_count", @"trip_time_in_secs", @"trip_distance" }))
		.Append(cvMLContext.Regression.Trainers.FastForest(labelColumnName: "fare_amount"));

使用 `CrossValidate` 方法开始使用定义的管道对您的数据进行训练和评估。 默认情况下，数据分为五个子集，但您可以使用“numberOfFolds”参数将其设置为您喜欢的任何值。

In [1]:
var cvResults = cvMLContext.Regression.CrossValidate(trainSet, cvMLPipeline, labelColumnName: "fare_amount");

In [1]:
cvResults.Select(x => x.Metrics)

index,MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
0,1.3080861318665928,10.437064100433819,3.230644533283385,10.437063966978595,0.8884018879298109
1,1.360165666717988,10.798742155503342,3.2861439645127146,10.798742212339462,0.8850517190709868
2,1.2518280553341605,8.216770508755435,2.8664909748253935,8.216770540499184,0.9086191464533752
3,1.3420262660438258,10.229870802662239,3.1984169213319014,10.229870780300226,0.8886983637741152
4,1.5430782453416307,10.122911443078698,3.1816523133552317,10.122911478082717,0.88835065401792


### 计算测试集评估指标

与前面的示例一样，在完整测试集上使用“评估”方法来评估使用交叉验证训练的模型的性能。

In [1]:
var cvTestEvalMetrics = 
	cvResults
		.Select(fold => fold.Model.Transform(trainTestData.TestSet))
		.Select(predictions => cvMLContext.Regression.Evaluate(predictions, "fare_amount", "Score"));

In [1]:
cvTestEvalMetrics

index,MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
0,1.2946164063640293,9.741100529118015,3.1210736180228134,9.741100436339362,0.8944318265394658
1,1.3517783571125017,10.697284288708751,3.270670311833455,10.697284293348838,0.8840692835505208
2,1.258738437439045,10.056973626481687,3.1712731869836897,10.056973680972417,0.8910085843878911
3,1.3321059289142654,10.119475327642087,3.1811122783771855,10.119475305281489,0.8903312286404623
4,1.5373861941022984,11.21522705311926,3.3489143096112897,11.215227048598013,0.878456132199452


## 模型可解释性

评估指标是量化模型对新数据的预测效果的好方法。但是，一个好的评估指标不应该是您在评估模型时考虑的唯一因素。为了对您的模型及其做出的决策建立更多的信任，了解它如何以及为什么做出它所做的决策非常重要。

模型在社会中变得越来越普遍，并且正在影响个人的生活。例如，假设使用机器学习模型进行医学诊断。该模型的诊断很可能是正确的，但由于对个人健康的影响，做出错误诊断的风险很高。因此，所有利益相关者（患者、医生、监管机构）都必须了解是什么促使模型做出诊断，以便对其决策充满信心。

### 全局和局部解释

在解释机器学习模型时，您可以在全局和本地级别进行。

全局解释是您在总体水平上进行概括。例如，假设您正在构建一个模型来预测出租车费用。在解释为什么某些票价比其他票价更贵时，您很可能会发现行程更长或时间更长的游乐设施可能更贵。尽管这并不能准确地告诉您为什么任何一种特定的游乐设施都比另一种更昂贵，但在总体层面上，您可以在模型做出决策时看到哪些特征对模型很重要。

如果您在解释模型的决策时需要更多粒度，这就是局部解释的用武之地。局部解释允许您查看任何单个预测，哪些特征有助于模型的决策。例如，假设一个模型用于确定个人贷款的信用风险。给定两个具有不同债务、收入和付款历史的客户，该模型确定哪个客户更有可能偿还贷款。使用本地可解释性技术，您可以在个人层面检查哪些特征促成了拒绝贷款的决定。

### ML.NET 中的可解释性技术

ML.NET 提供了两种解释模型的技术：

- 排列特征重要性（PFI）
- 特征贡献计算（FCC）

#### 排列特征重要性（PFI）

排列特征重要性是一种**全局**可解释性技术。在高层次上，它对整个数据集一次随机打乱数据一个特征，并计算感兴趣的性能指标降低了多少。变化越大，该特征就越重要。有关详细信息，请参阅 [使用排列特征重要性解释模型预测](https://docs.microsoft.com/dotnet/machine-learning/how-to-guides/explain-machine-learning-model-permutation-feature-importance -ml-net)

#### 特征贡献计算（FCC）

特征贡献计算是一种**本地**可解释性技术。该技术计算每个预测的每个特征贡献的模型特定列表。这些贡献可以是正面的（它们使评估指标更高）或负面的（它们使评估指标更低）。

### Explain models with Permutation Feature Importance (PFI)

#### Initialize MLContext

In [1]:
var pfiMLContext = new MLContext();

#### 定义数据准备管道

In [1]:
var pfiDataPipeline = 
	pfiMLContext.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair(@"vendor_id", @"vendor_id"), new InputOutputColumnPair(@"payment_type", @"payment_type")},outputKind: OutputKind.Binary)
		.Append(pfiMLContext.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair(@"rate_code", @"rate_code"), new InputOutputColumnPair(@"passenger_count", @"passenger_count"), new InputOutputColumnPair(@"trip_time_in_secs", @"trip_time_in_secs"), new InputOutputColumnPair(@"trip_distance", @"trip_distance") }))
        .Append(pfiMLContext.Transforms.Concatenate(@"Features", new[] { @"vendor_id", @"payment_type", @"rate_code", @"passenger_count", @"trip_time_in_secs", @"trip_distance" }));

#### 将数据转换应用于训练数据

In [1]:
var pfiPreprocessedData = 
	pfiDataPipeline
		.Fit(trainSet)
		.Transform(trainSet);

#### 定义训练器

In [1]:
var pfiTrainer = pfiMLContext.Regression.Trainers.FastForest(labelColumnName: "fare_amount");

#### 使训练器适配您的预处理数据

In [1]:
var pfiModel = pfiTrainer.Fit(pfiPreprocessedData);

#### 计算排列特征重要性（PFI）

In [1]:
var permutationFeatureImportance =
    mlContext
        .Regression
        .PermutationFeatureImportance(pfiModel, pfiPreprocessedData, permutationCount:3, labelColumnName: "fare_amount");

#### 提取 R 平方度量

In [1]:
var pfiMetrics = 
	permutationFeatureImportance
		.Select((metric,idx) => new {idx, metric.RSquared})
		.OrderByDescending(x => Math.Abs(x.RSquared.Mean));

#### 获取属性名称列表

In [1]:
var featureContributionColumn = pfiPreprocessedData.Schema.GetColumnOrNull("Features");
var slotNames = new VBuffer<ReadOnlyMemory<char>>();
featureContributionColumn.Value.GetSlotNames(ref slotNames);
var slotNameValues = slotNames.DenseValues();

#### 将 PFI 指标映射到特征名称

In [1]:
var featureImportance = 
	pfiMetrics
		.Zip(slotNameValues, (a,b) => new KeyValuePair<string,double>(b.ToString(),a.RSquared.Mean));

featureImportance

index,Key,Value
0,vendor_id.Bit2,-0.5103167076996584
1,vendor_id.Bit1,-0.20920573710951015
2,vendor_id.Bit0,-0.20524726245559932
3,payment_type.Bit3,-0.0013735609832817113
4,payment_type.Bit2,-0.0005371983420188927
5,payment_type.Bit1,-0.00015402329111213753
6,payment_type.Bit0,-7.256291485776185E-05
7,rate_code,-5.605438192501921E-07
8,passenger_count,0
9,trip_time_in_secs,0


### 使用特征贡献计算 (FCC) 解释模型

#### MLContext 初始化

In [1]:
var fccMLContext = new MLContext();

#### 定义数据准备管道

In [1]:
var fccDataPipeline = 
	fccMLContext.Transforms.Categorical.OneHotEncoding(new[] { new InputOutputColumnPair(@"vendor_id", @"vendor_id"), new InputOutputColumnPair(@"payment_type", @"payment_type")},outputKind: OutputKind.Binary)
		.Append(fccMLContext.Transforms.ReplaceMissingValues(new[] { new InputOutputColumnPair(@"rate_code", @"rate_code"), new InputOutputColumnPair(@"passenger_count", @"passenger_count"), new InputOutputColumnPair(@"trip_time_in_secs", @"trip_time_in_secs"), new InputOutputColumnPair(@"trip_distance", @"trip_distance") }))
        .Append(fccMLContext.Transforms.Concatenate(@"Features", new[] { @"vendor_id", @"payment_type", @"rate_code", @"passenger_count", @"trip_time_in_secs", @"trip_distance" }));

#### 将数据转换应用于训练数据

In [1]:
var fccPreprocessedData = 
	fccDataPipeline
		.Fit(trainSet)
		.Transform(trainSet);

#### 定义训练器

In [1]:
var fccTrainer = fccMLContext.Regression.Trainers.FastForest(labelColumnName: "fare_amount");

#### 使训练器适合您的预处理数据

In [1]:
var fccModel = fccTrainer.Fit(fccPreprocessedData);

#### 计算特征贡献

In [1]:
var featureContributionCalc = 
	fccMLContext.Transforms.CalculateFeatureContribution(fccModel,normalize:false)
		.Fit(fccPreprocessedData)
		.Transform(fccPreprocessedData);

#### 获取功能名称列表

In [1]:
var featureContributionColumn = featureContributionCalc.Schema.GetColumnOrNull("FeatureContributions");
var slotNames = new VBuffer<ReadOnlyMemory<char>>();
featureContributionColumn.Value.GetSlotNames(ref slotNames);
var slotNameValues = slotNames.DenseValues();

#### 获取特征贡献值

In [1]:
var featureContributionValues = featureContributionCalc.GetColumn<float[]>("FeatureContributions");

#### 使用特征名称映射特征贡献值

In [1]:
var featureContributions = 
	featureContributionValues
		.Select(x => x.Zip(slotNameValues, (a,b) => new KeyValuePair<string,float>(b.ToString(),a)));

#### 显示第一个预测贡献的特征

In [1]:
featureContributions.First()

index,Key,Value
0,vendor_id.Bit2,0
1,vendor_id.Bit1,0
2,vendor_id.Bit0,6.46816
3,payment_type.Bit3,0
4,payment_type.Bit2,-3.4585545
5,payment_type.Bit1,19.19871
6,payment_type.Bit0,10.600954
7,rate_code,-1779.6366
8,passenger_count,-1.6658905
9,trip_time_in_secs,75.95777


## **如何改进我的模型？**

模型评估是机器学习工作流程中的一个重要步骤，用于确定模型是否已准备好部署到生产中。如果您的模型不符合您的生产就绪标准，您可以尝试一些事情来改进您的模型。这些包括：

- **重新定义问题** - 您是否正在尝试解决正确的问题？考虑从不同的角度看待问题。
- **提供更多数据样本** - 经验是最好的老师。提供更多代表您的问题空间的示例有助于培训师识别更多边缘案例。
- **添加功能（更多上下文）** - 围绕数据点构建上下文有助于算法和主题专家更好地做出决策。例如，一所房子有三间卧室这一事实本身并不能很好地表明它的价格。但是，如果您添加上下文并且现在知道它位于主要大都市区以外的郊区，平均年龄为 38 岁，平均家庭收入为 80,000 美元，学校位于前 20 个百分位，那么该算法有更多信息可以作为其基础上的决定。
- **使用有意义的数据和特征** - 更多数据和特征（上下文）有助于提高准确性，但也会引入噪音。考虑使用置换特征重要性 (PFI) 和特征贡献计算 (FCC) 来确定影响您的预测的特征并删除任何对您的模型没有贡献的特征。
- **使用交叉验证** - 交叉验证可以成为训练具有较小数据集的模型的有效工具。
- **超参数调整** - 超参数与训练期间学习的参数一样重要。找到合适的人是一个反复试验的过程。使用 AutoML 搜索空间和全面估计器来帮助您为算法找到正确的超参数。
- **选择不同的算法** - 就像超参数调整一样，找到合适的算法来训练你的模型是一个反复试验的过程。使用 AutoML 帮助您遍历 ML.NET 中可用的各种算法，以帮助您选择最佳算法来解决您的问题。

有关详细信息，请参阅有关 [如何改进 ML.NET 模型] (https://docs.microsoft.com/dotnet/machine-learning/resources/improve-machine-learning-model-ml-net) 的指南。

## 进阶学习

> [⏪ 上一个学习模块 - 训练和AutoML](https://github.com/kinfeylo/csharp-notebooks/blob/main/machine-learning/03-Training%20and%20AutoML.ipynb)

### 更多实例

- [Binary Classification with Titanic Dataset]([ntbk.io/ml-ref-kaggle-titanic](https://github.com/kinfey/csharp-notebooks/blob/main/machine-learning/REF-Kaggle%20with%20Titanic%20Dataset.ipynb))  
- [Value Prediction/Regression with Taxi Dataset](https://ntbk.io/ml-e2e-taxi)  